In [ ]:
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.animation as animation

random.seed(67890)

In [ ]:
class BoxAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.box_carried = False

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def step(self):
        if not self.box_carried and self.model.is_box(self.pos):
            self.box_carried = True
            self.model.pick_box(self.pos)
        elif self.box_carried and self.model.count_boxes() < 5:
            self.box_carried = False
            self.model.place_box(self.pos)
        self.move()

In [ ]:
class WarehouseModel(Model):
    def __init__(self, width, height, num_agents, num_boxes):
        self.num_agents = num_agents
        self.num_boxes = num_boxes
        self.grid = MultiGrid(width, height, torus=False)
        self.schedule = RandomActivation(self)
        self.boxes = np.zeros((width, height))

        # Create agents
        for i in range(self.num_agents):
            a = BoxAgent(i, self)
            self.schedule.add(a)
            # Place the agent in a random position
            pos = self.random_position()
            while self.is_box(pos):
                pos = self.random_position()
            self.grid.place_agent(a, pos)

        # Place boxes
        self.place_boxes(num_boxes)

        # Create a data collector
        self.datacollector = DataCollector(
            model_reporters={"TotalBoxes": self.count_boxes, 
                             "CarriedBoxes": self.count_carried_boxes, "Grid": self.get_grid}
        )

    def place_boxes(self, num_boxes):
        for _ in range(num_boxes):
            # Choose a random position
            pos = self.random_position()
            # Add 1 to 3 boxes to the cell
            self.boxes[pos[0]][pos[1]] += random.randint(1, 3)

    def is_box(self, pos):
        return self.boxes[pos[0]][pos[1]] > 0

    def pick_box(self, pos):
        if self.is_box(pos):
            self.boxes[pos[0]][pos[1]] -= 1
            return 1
        return 0

    def place_box(self, pos):
        if self.boxes[pos[0]][pos[1]] < 5:
            self.boxes[pos[0]][pos[1]] += 1

    def count_boxes(self):
        return np.sum(self.boxes)

    def count_carried_boxes(self):
        return sum(agent.box_carried for agent in self.schedule.agents)

    def random_position(self):
        x = random.randint(0, self.grid.width - 1)
        y = random.randint(0, self.grid.height - 1)
        return (x, y)

    def is_simulation_done(self):
        for row in self.boxes:
            for stack in row:
                if stack != 0 and stack != 5:
                    return False
        return True

    def get_grid(self):
        return self.boxes.copy()

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()


In [ ]:

# Parameters
WIDTH = 20
HEIGHT = 20
NUM_AGENTS = 5
NUM_BOXES = 200